In [732]:
import os
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup

In [733]:
SCORES_DIR = "data/scores"
team_abbrev = pd.read_csv("teams.csv")

In [734]:
box_scores = os.listdir(SCORES_DIR)
box_scores = [os.path.join(SCORES_DIR, f) for f in box_scores if f.endswith(".html")]

In [735]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    
    # Removing headers that would interfere with pandas
    [s.decompose() for s in soup.select("caption")]
    [s.decompose() for s in soup.select(".filter")]
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    [s.decompose() for s in soup.select("img.teamlogo")]
    [s.decompose() for s in soup.select("div.media-item.logo")]
    [s.decompose() for s in soup.select("em")]
    [s.decompose() for s in soup.select("li.hasmore")]
    [s.decompose() for s in soup.select("div.section_heading_text")]
    [s.decompose() for s in soup.select("div.section_heading_text")]
    [s.decompose() for s in soup.select("div.callout")]
    [s.decompose() for s in soup.select("div.section_wrapper")]
    [s.decompose() for s in soup.select("div.adblock")]

    # Test for changes
    # html_snip = soup.prettify()
    # save_path = os.path.join(SCORES_DIR, "asdf.html")
    # with open(save_path, "w+", encoding="utf-8") as f:
    #     f.write(html_snip)
    
    return soup

In [736]:
def read_line_score(soup):
    html_str = str(soup)
    html_io = StringIO(html_str)
    
    line_score = pd.read_html(html_io, attrs={"class": "linescore"})[0]
    line_score = line_score.drop(line_score.columns[0], axis=1) # Drops extra NaN column
    cols = list(line_score.columns)
    cols[0] = "Tm"
    cols[-1] = "Final"
    line_score.columns = cols

    # Create a dictionary for mapping full team names to abbreviations
    team_abbrev_dict = dict(zip(team_abbrev["Team Name"], team_abbrev["Abbreviation"]))
    
    # Replace full team names with abbreviations in the "Team" column
    line_score["Tm"] = line_score["Tm"].map(team_abbrev_dict).fillna(line_score["Tm"])
    
    line_score = line_score[["Tm", "Final"]]
    return line_score

In [737]:
def read_stats(soup, team, stat):
    html_str = str(soup)
    html_io = StringIO(html_str)

    try:
        df = pd.read_html(html_io, attrs={"id": stat})[0]
    except ValueError:
        return None
        
    no_convert = ["Tm", "Player"]
    cols_to_convert = [col for col in df.columns if col not in no_convert]
    
    for col in cols_to_convert:
        if df[col].dtype == object:
            if df[col].str.contains('%').any():  
                df[col] = df[col].str.replace('%', '', regex=False)
                df[col] = df[col].astype(float)
                df[col] = pd.to_numeric(df[col], errors='coerce')
                
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors="coerce")
    
    df = df[df["Tm"] == team]
    
    if stat == "passing_advanced":
        df.columns = [
            f"Pass{col}" if col not in no_convert else col for col in df.columns
        ]
        total_columns = ["PassCmp", "PassAtt", "PassYds", "Pass1D", "PassIAY", "PassCAY", "PassYAC", "PassDrops", 
                         "PassBadTh", "PassSk", "PassBltz", "PassHrry", "PassHits", "PassScrm"]
    elif stat == "rushing_advanced":
        df.columns = [
            f"Rush{col}" if col not in no_convert else col for col in df.columns
        ]
        total_columns = ["RushAtt", "RushYds", "RushTD", "Rush1D", "RushYBC", "RushYAC", "RushBrkTkl"]
    elif stat == "receiving_advanced":
        df.columns = [
            f"Rec{col}" if col not in ["Tm", "Player", "Rec"] else col for col in df.columns
        ]
        total_columns = ["RecTgt", "Rec", "RecYds", "RecTD", "Rec1D", "RecYBC", 
                          "RecYAC", "RecBrkTkl", "RecDrop", "RecInt"]
    elif stat == "defense_advanced":
        df.columns = [
            f"Def{col}" if col not in no_convert else col for col in df.columns
        ]
        total_columns = ["DefInt", "DefTgt", "DefCmp", "DefYds", "DefTD", "DefAir", "DefYAC", 
                     "DefBltz", "DefHrry", "DefQBKD", "DefSk", "DefPrss", "DefComb", "DefMTkl"]
    elif stat == "returns":
         # Add a 'P' prefix to the second set of columns (punt stats)
        df.rename(columns={
            df.columns[2]: "KickRet",            
            df.columns[3]: "KickRetYds",         
            df.columns[4]: "KickRetY/Rt",        
            df.columns[5]: "KickRetTD",          
            df.columns[6]: "KickRetLng",         
            df.columns[7]: "PuntRet",
            df.columns[8]: "PuntRetYds",
            df.columns[9]: "PuntRetY/R",
            df.columns[10]: "PuntRetTD",
            df.columns[11]: "PuntRetLng"
        }, inplace=True)

        df.columns = df.columns.str.replace(r'\.1$', '', regex=True)

        total_columns = ["KickRet", "KickRetYds", "KickRetTD", "KickRetLng", "PuntRet", 
                         "PuntRetYds", "PuntRetTD", "PuntRetLng"]
    elif stat == "kicking":
        df.rename(columns={          
            df.columns[7]: "PuntYds",
            df.columns[8]: "PuntY/P",
            df.columns[9]: "PuntLng",
        }, inplace=True)
        total_columns = ["XPM", "XPA", "FGM", "FGA", "Pnt", "PuntYds", "PuntLng"]
    totals = df[total_columns].sum()

    # Add a new row with the totals
    totals_row = pd.DataFrame(totals).T
    totals_row["Tm"] = team
    totals_row["Player"] = "Totals"

    # Append the totals row to the df
    df = pd.concat([df, totals_row], ignore_index=False)

    return df

In [738]:
def read_season_info(soup):
    nav = soup.find("div", class_="scorebox_meta")
    date_text = nav.find("div").get_text(strip=True)

    year = date_text.split()[-1]
    
    return year

In [748]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["Tm"]) 
    
    summaries = []
    for team in teams:
        passing = read_stats(soup, team, "passing_advanced")
        rushing = read_stats(soup, team, "rushing_advanced")
        receiving = read_stats(soup, team, "receiving_advanced")
        defense = read_stats(soup, team, "defense_advanced")
        returns = read_stats(soup, team, "returns")
        kicking = read_stats(soup, team, "kicking")

        if any(stat is None for stat in [passing, rushing, receiving, defense, returns, kicking]):
            print(f"{box_score} - missing table")
            continue
    
        totals = pd.concat([passing.iloc[-1,:], rushing.iloc[-1,:], receiving.iloc[-1,:], 
                            defense.iloc[-1,:], returns.iloc[-1,:], kicking.iloc[-1,:]])
        totals.index = totals.index.str.lower()
        totals = totals[~totals.index.str.contains('tm|player', case=False)]
            
        maxes = pd.concat([passing.iloc[:-1,:].max(), rushing.iloc[:-1,:].max(), receiving.iloc[:-1,:].max(),
                           defense.iloc[:-1,:].max(), returns.iloc[:-1,:].max(), kicking.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        maxes = maxes[~maxes.index.str.contains('tm|player', case=False)]
        
        summary = pd.concat([totals, maxes])
        
        if base_cols == None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
    
        summary = summary[base_cols]
        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T
    
    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"
    
    full_game = pd.concat([game, game_opp], axis=1)
    
    full_game["season"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["Final"] > full_game["Final_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 207
200 / 207


In [749]:
games_df = pd.concat(games, ignore_index=True)

In [750]:
games_df

,passcmp,passatt,passyds,pass1d,pass1d%,passiay,passiay/pa,passcay,passcay/cmp,passcay/pa,...,pnt_max_opp,puntyds_max_opp,punty/p_max_opp,puntlng_max_opp,Tm_opp,Final_opp,home_opp,season,date,won
0,26,41,273,11,NaN,271,NaN,107,NaN,NaN,...,3.0,144.0,48.0,56.0,KAN,27,1,2024,2024-09-05,False
1,20,28,291,15,NaN,165,NaN,103,NaN,NaN,...,2.0,75.0,37.5,40.0,BAL,20,0,2024,2024-09-05,True
2,17.0,35.0,260.0,9.0,NaN,314.0,NaN,122.0,NaN,NaN,...,2.0,101.0,50.5,54.0,PHI,34,1,2024,2024-09-06,False
3,20.0,34.0,278.0,14.0,NaN,242.0,NaN,108.0,NaN,NaN,...,2.0,74.0,37.0,41.0,GNB,29,0,2024,2024-09-06,True
4,17,23,156,6,NaN,122,NaN,102,NaN,NaN,...,4.0,210.0,52.5,59.0,ATL,10,1,2024,2024-09-08,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,18,29,295,13,NaN,245,NaN,144,NaN,NaN,...,4.0,177.0,44.3,51.0,LVR,13,0,2024,2024-12-08,True
410,33,44,369,18,NaN,280,NaN,162,NaN,NaN,...,4.0,198.0,49.5,52.0,DAL,20,1,2024,2024-12-09,True
411,16,31,183,9,NaN,233,NaN,97,NaN,NaN,...,3.0,121.0,40.3,61.0,CIN,27,0,2024,2024-12-09,False
412,16,27,160,6,NaN,177,NaN,75,NaN,NaN,...,7.0,316.0,45.1,55.0,SFO,6,1,2024,2024-12-12,True


In [751]:
games_df.to_csv("2024_nfl_games.csv")